In [1]:
# Imports
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
import requests
from am_sma_data_cleaning.utils import remove_spammers
from am_sma_data_cleaning.utils import count_column_tokens
import glob
import re
from tqdm.notebook import tqdm
import unicodedata
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.interpolate import make_interp_spline
import matplotlib.ticker as mtick
import dataframe_image as dfi
#--
import fasttext
from langdetect import detect
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm

### Helper Functions And Global Variables

In [2]:
CURRENT_DATE_TIME = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

def normalize_post(text):
    """
    Convert the text to lowercase and remove URLs, hashtags, mentions, punctuation,
    and extra whitespace.
    """
    if not isinstance(text, str):
        # Convert non-string (like float NaN) to an empty string
        text = ""
    text = text.lower()
    #text = re.sub(r'http\S+', '', text)   # Remove URLs
    text = re.sub(
        r'(?:https?://|www\.)\S+|'        # http:// or https:// or www.
        r'\b\S+\.(?:com|org|net|io|co|gov|edu)(?:/\S*)?',  # domain.tld[/…]
        '',
        text
    )
    #text = re.sub(r'[@#]\S+', '', text)     # Remove hashtags and mentions
    #text = re.sub(r'[^\w\s]', '', text)     # Remove punctuation
    return text.strip()

def tokenize(text):
    """
    Simple word-based tokenization.
    You could also use n-grams for more robust matching.
    """
    return text.split()

In [ ]:
def identify_nonorganic_activity(
    df: pd.DataFrame,
    *,
    post_rate_denominator: float = 50,
    risk_threshold: float = 0.60,
    show_progress: bool = True
) -> pd.DataFrame:
    """
    Returns an inspection table of accounts exhibiting automated,
    promotional, or otherwise non-organic posting behaviour. It returns a
    per-author inspection table (`bot_df`).
    Parameters
    ----------
    df : pandas.DataFrame
        **Tweet-level** records.  Required columns (case-sensitive):

        * 'Author ID'         – unique account identifier  
        * 'Date'              – timestamp; will be converted with `pd.to_datetime`  
        * 'Post Text'         – tweet text (str)  
        * 'Total Engagement'  – numeric (likes + RTs + …)

    post_rate_denominator : float, default=50
        Denominator used to scale the posting-rate feature  
        :math:`f1 = \\min\\bigl(\\tfrac{\\text{tweets_per_day}}
        {\\text{post_rate_denominator}}, 1\\bigr)`.  
        Larger values make *f1* less sensitive to heavy posting.

    bot_threshold : float in [0, 1], default=0.60
        Accounts whose composite *bot_score* ≥ `bot_threshold`
        are returned for inspection.

    show_progress : bool, default=True
        If *True*, activates a TQDM progress-bar when scanning tweets for URLs.

    Returns
    -------
    bot_df : pandas.DataFrame
        Author-level table sorted by descending `bot_score`, with columns

        * 'Author ID' (index reset to column)  
        * 'tweets_total', 'tweets_per_day'  
        * 'f1_posting', 'f2_url_ratio', 'f3_dup_ratio', 'f4_low_eng'  
        * 'bot_score'

        Rows are limited to authors whose score meets the threshold.

    Notes
    -----
    * **No in-place mutation** – the original `df` is not altered.
    * The feature set and equal-weight scoring scheme are deliberately
      simple and transparent for experimental work (e.g. doctoral
      hypothesis testing).  Swap in additional features or different
      weighting if needed.
    """

    # ------------------------------------------------------------------
    # 0.  Preparations
    # ------------------------------------------------------------------
    if show_progress:
        tqdm.pandas()
        apply = lambda s, func: s.progress_apply(func)
    else:
        apply = lambda s, func: s.apply(func)

    # shallow copy to avoid side-effects
    tweets = df.copy(deep=False)
    tweets['Date'] = pd.to_datetime(tweets['Date'], errors='coerce')

    # ------------------------------------------------------------------
    # 1.  Row-level helper flags
    # ------------------------------------------------------------------
    url_regex = re.compile(r"https?://", re.IGNORECASE)

    def _contains_url(text: str) -> bool:
        return bool(url_regex.search(str(text)))

    tweets['_has_url'] = apply(tweets['Post Text'], _contains_url)

    tweets['_is_duplicate'] = (
        tweets['Post Text']
        .str.lower()
        .str.strip()
        .duplicated(keep=False)
    )

    tweets['_is_low_eng'] = tweets['Total Engagement'].fillna(0) == 0

    # ------------------------------------------------------------------
    # 2.  Author-level aggregation
    # ------------------------------------------------------------------
    g = tweets.groupby('Author ID', dropna=False)

    agg = pd.DataFrame({
        'tweets_total'    : g.size(),
        'first_date'      : g['Date'].min(),
        'last_date'       : g['Date'].max(),
        'tweets_with_url' : g['_has_url'].sum(),
        'dup_tweets'      : g['_is_duplicate'].sum(),
        'low_eng_tweets'  : g['_is_low_eng'].sum()
    })

    # ------------------------------------------------------------------
    # 3.  Derived metrics & feature scaling
    # ------------------------------------------------------------------
    active_days = (agg['last_date'] - agg['first_date']).dt.days.clip(lower=1)
    agg['tweets_per_day'] = agg['tweets_total'] / active_days

    agg['f1_posting']   = (agg['tweets_per_day'] / post_rate_denominator).clip(upper=1)
    agg['f2_url_ratio'] = agg['tweets_with_url'] / agg['tweets_total']
    agg['f3_dup_ratio'] = agg['dup_tweets']      / agg['tweets_total']
    agg['f4_low_eng']   = agg['low_eng_tweets']  / agg['tweets_total']

    features = ['f1_posting', 'f2_url_ratio', 'f3_dup_ratio', 'f4_low_eng']
    agg['bot_score'] = agg[features].mean(axis=1)

    # ------------------------------------------------------------------
    # 4.  Filter & presentation
    # ------------------------------------------------------------------
    #bot_ids = agg.query('bot_score >= @bot_threshold').index
    bot_ids = agg.query('bot_score >= @risk_threshold').index

    display_cols = [
        'tweets_total',
        'tweets_per_day',
        'f1_posting', 'f2_url_ratio', 'f3_dup_ratio', 'f4_low_eng',
        'bot_score'
    ]

    bot_df = (
        agg.loc[bot_ids, display_cols]
           .sort_values('bot_score', ascending=False)
           .reset_index()          # keep 'Author ID' as a column
    )

    return bot_df
    

### Read All Data

In [4]:
##-- Use glob to find all CSV files recursively under the given directory
all_csv_files = glob.glob(
    "/home/shola/research_raw_data/**/*.csv",
    recursive=True
)
##-- Read each CSV file and store in a list
df_list = [pd.read_csv(csv_file) for csv_file in all_csv_files]

##-- Concatenate all into one DataFrame
df = pd.concat(df_list, ignore_index=True)

##-- Drop views since its not tracked by tweeter for later years
df = df.drop(columns="Views")

##-- Add column for total engagement
df["Total Engagement"] = (
    df["Replies"] + df["Likes"] + df["Reshares"]
)

##-- Add tweet id column
df["tweet_id"] = df["Author ID"].astype(str) + "_" + df["Date"].astype(str)

##--
df["Date"] = pd.to_datetime(df["Date"], format='ISO8601')

##-- Drop unwanted years..these must have been added via over run of the years..they are not full years but only few days in 
df = df[~df["Date"].dt.year.isin([2011, 2024, 2025])]  

#create column row_num
df = df.assign(row_num=range(1, len(df) + 1))

pd.set_option('display.max_colwidth', None)

In [5]:
df

,Author Name,Author ID,Date,Post Text,Replies,Likes,Reshares,Total Engagement,tweet_id,row_num
0,₊˚⊹♡ Ashley ୨୧,@Whimsu,2013-01-01 00:19:40+00:00,My sister got me this 3D printed gyroid for Christmas! Sooo cute!,0,0,0,0,@Whimsu_2013-01-01 00:19:40+00:00,1
1,Jon Witts,@jonwitts,2012-12-30 20:18:03+00:00,"This is just so cool! "" @josiefraser : 3D printer converts digital audio files into 33rpm records http:// """,0,0,0,0,@jonwitts_2012-12-30 20:18:03+00:00,2
2,Marshall Advanced Manufacturing Center,@MUAdvMfg,2012-12-30 15:13:18+00:00,3D printing predictions for 2013 - http:// 229-3d-printing-predictions-for-2013.html …,0,0,0,0,@MUAdvMfg_2012-12-30 15:13:18+00:00,3
3,tttitoflip,@tttitoflip,2012-12-30 13:19:02+00:00,Features/ 3D Printing http://,0,0,0,0,@tttitoflip_2012-12-30 13:19:02+00:00,4
4,3DPrint_news,@3dprint_news,2012-12-30 22:37:16+00:00,RT @RicardoBlanco This is the best analysis of the pros and cons of commerical-scale 3D printing I've seen. http:// -achilles-heel-of-3d-printing-015281 …,0,0,0,0,@3dprint_news_2012-12-30 22:37:16+00:00,5
...,...,...,...,...,...,...,...,...,...,...
6728757,"Helfrich Bicycles, LLC",@HelfrichBicycle,2022-12-17 04:56:50+00:00,"I custom built this wheel with a 3D printed logo for the @pmmarchingunit @pennmanorschools annual banquet basket raffle. #bicyclerepair #bicyclerestoration @ Lancaster, Pennsylvania",0,0,0,0,@HelfrichBicycle_2022-12-17T04:56:50.000Z,6725981
6728758,Ghost Guns,@buyghostguns,2022-12-17 01:21:51+00:00,SG22 .22LR 3D PRINTED BUILD KIT W/CMMG PARTS https:// 2lr-3d-printed-build-kit-w-cmmg-parts/?utm_source=ReviveOldPost&utm_medium=social&utm_campaign=ReviveOldPost … #22lr #pistols,0,0,0,0,@buyghostguns_2022-12-17T01:21:51.000Z,6725982
6728759,Smoke & Mirrors,@DrMask12,2022-12-16 23:21:22+00:00,"Finally decided to try and 3d print the B.P.S logo. Need to tweak some settings and make it a little larger, but I'd say it's a great success. I made the 3d model of the logo, credit for the actual digital logo goes to @alexkisterr",0,13,0,13,@DrMask12_2022-12-16T23:21:22.000Z,6725983
6728760,X Æ A-12,@ray5ar,2022-12-17 10:57:20+00:00,More 3D Printed Compliant Mechanisms https:// EIPA?feature=share … via @YouTube,0,0,0,0,@ray5ar_2022-12-17T10:57:20.000Z,6725984


In [6]:

# build a mask for “http:// ” or “https:// ” (scheme + space)
mask = df['Post Text'].str.contains(r'https?://\s', regex=True, na=False)

# filter the DataFrame
df_malformed_urls = df[mask]

df_malformed_urls[['Post Text']]


,Post Text
1,"This is just so cool! "" @josiefraser : 3D printer converts digital audio files into 33rpm records http:// """
2,3D printing predictions for 2013 - http:// 229-3d-printing-predictions-for-2013.html …
4,RT @RicardoBlanco This is the best analysis of the pros and cons of commerical-scale 3D printing I've seen. http:// -achilles-heel-of-3d-printing-015281 …
6,Infographic: 3D Printing and the Future of Manufacturing by Sculpteo http:// #3dprinting
7,RT @3DersOrg 3D printing predictions for 2013 http:// 229-3d-printing-predictions-for-2013.html … #3dprinting #3dprinter
...,...
6728750,"#3DPrint Tt Longship 3D Print https:// \nconsisting of a viking longship style, can be used as objective, scenery etc. ,excellent for all tabletop games scale 25 - 30 mm like D&D, Pathfinder,Warhammer etc. 3d printed with FDM printer, medium resolution"
6728755,"78% OFF,389.75 Original TEVOUP HYDRA 3D Printer Laser Engraver,free shipping https:// %c2%80389-75-original-tevoup-hydra-3d-printer-laser-engraverfree-shipping/ …"
6728758,SG22 .22LR 3D PRINTED BUILD KIT W/CMMG PARTS https:// 2lr-3d-printed-build-kit-w-cmmg-parts/?utm_source=ReviveOldPost&utm_medium=social&utm_campaign=ReviveOldPost … #22lr #pistols
6728760,More 3D Printed Compliant Mechanisms https:// EIPA?feature=share … via @YouTube


# Data Cleaning

## 1. Detect spam, bot-like, promotional, and other non-organic accounts  
Call to the helper function to score every author for automated or commercially driven behaviour and return `bot_df`, a 
table of those whose composite risk score meets or exceeds the specified threshold.


In [7]:
bot_df = identify_nonorganic_activity(
    df,
    post_rate_denominator=50,  # treat ≥ 50 tweets/day as “high”
    risk_threshold=0.75,       # flag accounts scoring ≥ 0.75 # this is an experimental value
    show_progress=True         # show tqdm progress bar
)

100%|█████████████████████████████████████████████████████████████████████████| 6725985/6725985 [00:09<00:00, 716090.19it/s]
/tmp/ipykernel_938/381492831.py:86: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  tweets['_is_low_eng'] = tweets['Total Engagement'].fillna(0) == 0


In [8]:
# sanity check
bot_df

,Author ID,tweets_total,tweets_per_day,f1_posting,f2_url_ratio,f3_dup_ratio,f4_low_eng,bot_score
0,@pashpatiluvsall,96,96.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,@AntoniaW4,1579,78.950000,1.000000,1.000000,0.998100,1.000000,0.999525
2,@DorothyU8,2070,103.500000,1.000000,1.000000,0.998068,1.000000,0.999517
3,@NancyN05,1413,58.875000,1.000000,1.000000,0.997169,1.000000,0.999292
4,@KaylinA17,3442,172.100000,1.000000,1.000000,0.996223,1.000000,0.999056
...,...,...,...,...,...,...,...,...
210043,@ACGGlobal,2,0.000563,0.000011,1.000000,1.000000,1.000000,0.750003
210044,@MelissaShirley2,2,0.000560,0.000011,1.000000,1.000000,1.000000,0.750003
210045,@akuli,2,0.000560,0.000011,1.000000,1.000000,1.000000,0.750003
210046,@AmyZ37,10,5.000000,0.100000,1.000000,0.900000,1.000000,0.750000


### Review Tweets Per User Over Certain Threshold (e.g. 1100) Per Year
This means how many duplicates tweets per year for author id

In [9]:
# Ensure Date is datetime
df_selected = df
df_selected['Date'] = pd.to_datetime(df['Date'])

# Extract year
df_selected['Year'] = df_selected['Date'].dt.year

# Group by Author ID, Normalized Text, and Year
duplicate_counts = (
    df_selected.groupby(['Author ID', 'Post Text', 'Year'])
    .size()
    .reset_index(name='tweet_repeat_count')
)

# Keep only repeated tweets within the same year
repeated_tweets = duplicate_counts[duplicate_counts['tweet_repeat_count'] > 1]

# Count how many *distinct* repeated tweets each user has per year
user_repeat_stats = (
    repeated_tweets.groupby(['Author ID', 'Year'])
    .size()
    .reset_index(name='num_repeated_texts')
)

# ilter for users who repeat >=50 messages in a single year
likely_bots = user_repeat_stats[user_repeat_stats['num_repeated_texts'] >= 1100]

# Sort by most suspicious activity
likely_bots = likely_bots.sort_values(by='num_repeated_texts', ascending=False)

likely_bots


,Author ID,Year,num_repeated_texts
55642,@buy3d,2014.0,6461
21212,@Hertz_ie,2017.0,3546
1515,@3d_printing_jp,2013.0,2239
21211,@Hertz_ie,2016.0,2065
54871,@bmine3rz,2016.0,1982
54872,@bmine3rz,2017.0,1933
54869,@bmine3rz,2014.0,1873
12980,@Darkspiral_94,2018.0,1869
12979,@Darkspiral_94,2017.0,1764
19046,@GaryWeston11,2018.0,1732


In [10]:
likely_bots_with_bot_score = (
    likely_bots
      .merge(bot_df[['Author ID','bot_score']],
             on='Author ID',
             how='left')
      .sort_values('bot_score', ascending=False)
      .reset_index(drop=True)
)
likely_bots_with_bot_score

,Author ID,Year,num_repeated_texts,bot_score
0,@buy3d,2014.0,6461,0.979537
1,@Hertz_ie,2017.0,3546,0.910365
2,@Hertz_ie,2016.0,2065,0.910365
3,@3d_printing_jp,2013.0,2239,0.902615
4,@3d_printing_jp,2014.0,1576,0.902615
5,@GaryWeston11,2018.0,1732,0.819955
6,@GaryWeston11,2017.0,1491,0.819955
7,@Darkspiral_94,2018.0,1869,0.790145
8,@Darkspiral_94,2017.0,1764,0.790145
9,@Darkspiral_94,2016.0,1176,0.790145


### Sanity Checks To Confirm Some of The Identified Non-Organic Accounts

In [11]:
# @bmine3rz

filtered = df.loc[df['Author ID'] == '@bmine3rz', ['Author ID','Post Text', 'Total Engagement']]
filtered

,Author ID,Post Text,Total Engagement
6,@bmine3rz,Infographic: 3D Printing and the Future of Manufacturing by Sculpteo http:// #3dprinting,0
7,@bmine3rz,RT @3DersOrg 3D printing predictions for 2013 http:// 229-3d-printing-predictions-for-2013.html … #3dprinting #3dprinter,0
64,@bmine3rz,RT @joanneprobyn Wired's Chris Anderson @chr1sa on the 3D Printing Revolution http:// via @foratv #3dprinting #3d #print ...,0
72,@bmine3rz,RT @igrowmogulsi The USB's with the 3D printing useful info and links on are in the post for last weeks delegates - happy browsing #3...,0
164,@bmine3rz,RT @3dprinting A Plea for More 3D Printer Manufacturers - Fabbaloo Blog - via @Fabbaloo - Daily News on 3D Printing : http:// 8/a-plea-for-more-3d-printer-manufacturers.html#.UN_5319yP-q.twitter …,0
...,...,...,...
6497277,@bmine3rz,Oculus Quest 2 3d Printed Controller Wall Mount https:// lus-quest-2-3d-printed-controller-wall-mount-2/ … #oculus#vr,0
6561400,@bmine3rz,3D printed Oculus Quest 2 rifle/gunstock Kit https:// printed-oculus-quest-2-rifle-gunstock-kit/ … #oculus#vr,0
6561437,@bmine3rz,3D printed Oculus Quest 2 noob rifle/pistol gunstock Kit https:// printed-oculus-quest-2-noob-rifle-pistol-gunstock-kit/ … #oculus#vr,0
6580744,@bmine3rz,KIWI design Deluxe Audio Strap Adapter Kit for Oculus/Meta Quest 2 (Not 3D Print https:// i-design-deluxe-audio-strap-adapter-kit-for-oculus-meta-quest-2-not-3d-print/ … #oculus#vr,0


In [12]:
##--- The DILEMA
# @bmine3rz # not really a bot perse but mostly not relevant tweets
# @Domain_Buyer #certainly a bot
# @3d_printing_jp
authors = ['@bmine3rz', '@Domain_Buyer', '@3d_printing_jp']
filtered = bot_df.loc[bot_df['Author ID'].isin(authors)].reset_index(drop=True)
filtered

,Author ID,tweets_total,tweets_per_day,f1_posting,f2_url_ratio,f3_dup_ratio,f4_low_eng,bot_score
0,@3d_printing_jp,20822,158.946565,1.000000,0.999376,0.611084,1.000000,0.902615
1,@Domain_Buyer,32123,22.432263,0.448645,0.850605,0.995330,0.953055,0.811909
2,@bmine3rz,102971,27.837524,0.556750,0.989482,0.594789,0.976314,0.779334


In [13]:
# total number of tweets across all rows in bot_df
total_tweets = bot_df['tweets_total'].sum()
print(total_tweets)


1034174


### 1.1 Remove zero-engagement tweets based on bot score algorithm 
Scrub out tweets that meet **both** of these conditions:
1. **Bot score**   
2. **No engagement** – `Total Engagement` is `0` (no likes, replies, or retweets).

In [14]:
# keep rows unless BOTH conditions are true:
#   1) the author is listed in bot_df
#   2) that row’s Total Engagement is 0
df = df[~((df['Author ID'].isin(bot_df['Author ID'])) & (df['Total Engagement'] == 0))]
df

,Author Name,Author ID,Date,Post Text,Replies,Likes,Reshares,Total Engagement,tweet_id,row_num,Year
0,₊˚⊹♡ Ashley ୨୧,@Whimsu,2013-01-01 00:19:40+00:00,My sister got me this 3D printed gyroid for Christmas! Sooo cute!,0,0,0,0,@Whimsu_2013-01-01 00:19:40+00:00,1,2013.0
1,Jon Witts,@jonwitts,2012-12-30 20:18:03+00:00,"This is just so cool! "" @josiefraser : 3D printer converts digital audio files into 33rpm records http:// """,0,0,0,0,@jonwitts_2012-12-30 20:18:03+00:00,2,2012.0
2,Marshall Advanced Manufacturing Center,@MUAdvMfg,2012-12-30 15:13:18+00:00,3D printing predictions for 2013 - http:// 229-3d-printing-predictions-for-2013.html …,0,0,0,0,@MUAdvMfg_2012-12-30 15:13:18+00:00,3,2012.0
3,tttitoflip,@tttitoflip,2012-12-30 13:19:02+00:00,Features/ 3D Printing http://,0,0,0,0,@tttitoflip_2012-12-30 13:19:02+00:00,4,2012.0
4,3DPrint_news,@3dprint_news,2012-12-30 22:37:16+00:00,RT @RicardoBlanco This is the best analysis of the pros and cons of commerical-scale 3D printing I've seen. http:// -achilles-heel-of-3d-printing-015281 …,0,0,0,0,@3dprint_news_2012-12-30 22:37:16+00:00,5,2012.0
...,...,...,...,...,...,...,...,...,...,...,...
6728757,"Helfrich Bicycles, LLC",@HelfrichBicycle,2022-12-17 04:56:50+00:00,"I custom built this wheel with a 3D printed logo for the @pmmarchingunit @pennmanorschools annual banquet basket raffle. #bicyclerepair #bicyclerestoration @ Lancaster, Pennsylvania",0,0,0,0,@HelfrichBicycle_2022-12-17T04:56:50.000Z,6725981,2022.0
6728758,Ghost Guns,@buyghostguns,2022-12-17 01:21:51+00:00,SG22 .22LR 3D PRINTED BUILD KIT W/CMMG PARTS https:// 2lr-3d-printed-build-kit-w-cmmg-parts/?utm_source=ReviveOldPost&utm_medium=social&utm_campaign=ReviveOldPost … #22lr #pistols,0,0,0,0,@buyghostguns_2022-12-17T01:21:51.000Z,6725982,2022.0
6728759,Smoke & Mirrors,@DrMask12,2022-12-16 23:21:22+00:00,"Finally decided to try and 3d print the B.P.S logo. Need to tweak some settings and make it a little larger, but I'd say it's a great success. I made the 3d model of the logo, credit for the actual digital logo goes to @alexkisterr",0,13,0,13,@DrMask12_2022-12-16T23:21:22.000Z,6725983,2022.0
6728760,X Æ A-12,@ray5ar,2022-12-17 10:57:20+00:00,More 3D Printed Compliant Mechanisms https:// EIPA?feature=share … via @YouTube,0,0,0,0,@ray5ar_2022-12-17T10:57:20.000Z,6725984,2022.0


In [15]:
KNOWN_POSSIBLE_BOT_OR_SPAM_ACCOUNTS = [
    "@weitingforyou",
    "@JayEfikeco",
    "@buy3d",
    "@GeekGawk",
    #"@bmine3rz",
    "@Reksaurian",
    "@Trend_deal_420",
    "@zeppy_3dprint",
    "@bubbleistrouble",
    "@HakanFagnell",
    "@Domain_Buyer",
    "@TriciaClyne",
    "@NEWsDealz",
    "@Deals_New_712",
    "@Deal_HOT_341",
    "@RealistRider",
    "@ShantelW6",
    "@SomeDroidCom",
    "@Arc_Deals_8812",
    "@Buy_Now_811",
    "@Best_Tips_443",
    "@truthinautism",
    "@Tweet_Deal_819",
    "@New_deal_932",
    "@My3DPrinting",
    "@Manhal_Deal_311",
    "@new_offer_827",
    "@danowall",
    "@Hot_Tips_123",
    "@lovepornsites",
    "@my3dsupplies",
    "@ISG3D",
    "@csmlibrarian",
    "@Great_Deal_691",
    "@bitcoinagile",
    #confirmed
    "@bectcomputing",
    "@Reksaurian",
    "@JohnTerz",
    "@AVS3DPrint",
    "@3dprintingjobs",
    "@SMilloow",
    "@PECHOLATATV", #irrelevant content mostly but not spam/bot account
]

In [16]:
# ── 1. Normalise the bot list in-place: lowercase + deduplicate ────────────────
KNOWN_POSSIBLE_BOT_OR_SPAM_ACCOUNTS = {
    aid.lower() for aid in KNOWN_POSSIBLE_BOT_OR_SPAM_ACCOUNTS
}  # now a set of unique, lower-cased IDs

# ── 2. Lower-cased, unique Author IDs from df ──────────────────────────────────
author_ids = set(df['Author ID'].astype(str).str.lower())

# ── 3. Find overlap ───────────────────────────────────────────────────────────
overlap_ids = KNOWN_POSSIBLE_BOT_OR_SPAM_ACCOUNTS & author_ids

# ── 4. Report sizes ───────────────────────────────────────────────────────────
print("unique bot IDs :", len(KNOWN_POSSIBLE_BOT_OR_SPAM_ACCOUNTS))
print("overlap count  :", len(overlap_ids))
print("overlap IDs    :", list(overlap_ids))


unique bot IDs : 40
overlap count  : 40
overlap IDs    : ['@new_deal_932', '@newsdealz', '@my3dsupplies', '@johnterz', '@geekgawk', '@manhal_deal_311', '@great_deal_691', '@deal_hot_341', '@isg3d', '@arc_deals_8812', '@realistrider', '@bectcomputing', '@best_tips_443', '@jayefikeco', '@pecholatatv', '@buy3d', '@csmlibrarian', '@zeppy_3dprint', '@deals_new_712', '@bitcoinagile', '@smilloow', '@bubbleistrouble', '@domain_buyer', '@tweet_deal_819', '@somedroidcom', '@weitingforyou', '@buy_now_811', '@shantelw6', '@hakanfagnell', '@new_offer_827', '@danowall', '@my3dprinting', '@reksaurian', '@lovepornsites', '@3dprintingjobs', '@trend_deal_420', '@hot_tips_123', '@triciaclyne', '@truthinautism', '@avs3dprint']


#### 1.2 Remove Tweets From Suspected Bot/Promo/Non-relevant Content Account
Remove only tweets from suspected (this was observed during data collection from tweeter UI) bot accounts *if* they have zero total engagement. This helps reduce noise from automated or spammy accounts without discarding possibly valuable high-engagement posts.

In [17]:
df = df[~(
    df['Author ID'].astype(str).str.lower().isin(KNOWN_POSSIBLE_BOT_OR_SPAM_ACCOUNTS)
    & (df['Total Engagement'] == 0)
)]

In [18]:
#@buy3d
#@bubbleistrouble
filtered = df.loc[df['Author ID'] == '@bubbleistrouble', ['Author ID','Post Text', 'Total Engagement']]
filtered

,Author ID,Post Text,Total Engagement
358937,@bubbleistrouble,"3D printed cap makes many, slimy bubbles",2
410719,@bubbleistrouble,"3D printed trash can makes big, juicy bubbles",2
410739,@bubbleistrouble,"3D printed trash can makes ridiculously, ephemeral bubbles",2
463955,@bubbleistrouble,"3D printed trash can makes ridiculously, ephemeral bubbles",2
499916,@bubbleistrouble,"3D printed trash can makes ridiculously, shinny bubbles",2
499988,@bubbleistrouble,"3D printed cap makes many, slimy bubbles",2
507294,@bubbleistrouble,"3D printed trash can makes many, reflective bubbles",1
526162,@bubbleistrouble,3D printed trash can makes abundent ephemeral bubbles,2
542531,@bubbleistrouble,"3D printed cap makes many, fleeting bubbles",2
577875,@bubbleistrouble,"3D printed cap makes fat, reflective bubbles",2


In [19]:
# Remove accounts that are certainly bot/spam/sales/promo accounts even though their tweets has engagements
to_remove = ['@bubbleistrouble', '@All_In_Comm']
df = df.loc[~df['Author ID'].isin(to_remove)].reset_index(drop=True)

## 2. Remove Tweets From Accounts With Promotional or Spam-Like Usernames
NOTE: Many promotional or spam accounts include keywords like "buy", "deal", "sale", etc. in their usernames.
This pattern used in the python code belowtargets such accounts by matching those keywords at word boundaries (start, end, or surrounded by underscores).
Tweets from these accounts are removed to reduce noise in the dataset.

In [20]:
pattern = r'(deal|buy|sale|discount|offer|promo|bargain)_'
# Using case-insensitive flag instead of .str.lower()
promo_zero_engagement = df['Author ID'].str.strip().str.contains(pattern, flags=re.IGNORECASE, na=False) & (df['Total Engagement'] == 0)
promo_account_tweets_df = df[promo_zero_engagement]

# filter them out
df = df[~promo_zero_engagement]

#Show for review
promo_account_tweets_df

/tmp/ipykernel_938/2672724651.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  promo_zero_engagement = df['Author ID'].str.strip().str.contains(pattern, flags=re.IGNORECASE, na=False) & (df['Total Engagement'] == 0)


,Author Name,Author ID,Date,Post Text,Replies,Likes,Reshares,Total Engagement,tweet_id,row_num,Year
7271,Marketing Madness,@Promo_Madness,2012-12-16 15:10:42+00:00,Interactive 3D Printed LED Pixel Hat Simulates Gravity: Advertise here with BSAWhen it com... http:// http://,0,0,0,0,@Promo_Madness_2012-12-16 15:10:42+00:00,8957,2012.0
8400,Marketing Madness,@Promo_Madness,2012-12-14 19:55:18+00:00,Creating Simple Electronics Through 3D Printing Is Now Possible: Advertise here with BSA3D... http:// http://,0,0,0,0,@Promo_Madness_2012-12-14 19:55:18+00:00,10447,2012.0
27553,Marketing Madness,@Promo_Madness,2012-11-22 19:06:51+00:00,DIY 3D Printed iPhone Case (Even Incorporate Your Favorite Sound): Advertise here with BSA... http:// http://,0,0,0,0,@Promo_Madness_2012-11-22 19:06:51+00:00,34318,2012.0
32496,Marketing Madness,@Promo_Madness,2012-11-16 19:03:07+00:00,Get Your Own Minecraft World 3D Printed & Make It All Real: Advertise here with BSAAs we t... http:// http://,0,0,0,0,@Promo_Madness_2012-11-16 19:03:07+00:00,40676,2012.0
41194,Marketing Madness,@Promo_Madness,2012-10-29 23:40:36+00:00,3D Printing Guide: How It All Works [Infographic]: \nAdvertise here with BSAImagine a wo... http:// http://,0,0,0,0,@Promo_Madness_2012-10-29 23:40:36+00:00,51506,2012.0
...,...,...,...,...,...,...,...,...,...,...,...
5321271,buy_silver,@buy_silver,2022-11-02 22:36:20+00:00,My first stack in a 3D printed container #silver,0,0,0,0,@buy_silver_2022-11-02T22:36:20.000Z,6450058,2022.0
5326770,buy_silver,@buy_silver,2022-08-02 16:36:41+00:00,"Pure Silver Mew & Pika, casted with 3d printed lost resin wax #silver",0,0,0,0,@buy_silver_2022-08-02T16:36:41.000Z,6456161,2022.0
5346996,The Bingo Deal,@deal_bingo,2022-08-16 06:30:30+00:00,3D Printing Pen With USB\n________________ #toys #kids #toys4life #toycommunity #deal #style #bestshopping #shoppingdaily #worldsales #shopping #onlineshopping https:// n-with-usb/ …,0,0,0,0,@deal_bingo_2022-08-16T06:30:30.000Z,6478491,2022.0
5352774,The Bingo Deal,@deal_bingo,2022-06-21 09:18:43+00:00,3D Printing Pen With USB\n________________\n\n #accessories #thebingodeal #sale #shopping https:// n-with-usb/ …,0,0,0,0,@deal_bingo_2022-06-21T09:18:43.000Z,6484875,2022.0


In [21]:
user_list = list(set(u.strip().strip('"') for u in KNOWN_POSSIBLE_BOT_OR_SPAM_ACCOUNTS if u.startswith('@')))
filtered_authors = promo_account_tweets_df['Author ID'].astype(str).str.strip().str.lower().unique()
not_in_filtered = [u for u in user_list if u.lower() not in filtered_authors]
not_in_filtered

['@new_deal_932',
 '@newsdealz',
 '@my3dsupplies',
 '@johnterz',
 '@geekgawk',
 '@manhal_deal_311',
 '@great_deal_691',
 '@deal_hot_341',
 '@isg3d',
 '@arc_deals_8812',
 '@realistrider',
 '@bectcomputing',
 '@best_tips_443',
 '@jayefikeco',
 '@pecholatatv',
 '@buy3d',
 '@csmlibrarian',
 '@zeppy_3dprint',
 '@deals_new_712',
 '@bitcoinagile',
 '@smilloow',
 '@bubbleistrouble',
 '@domain_buyer',
 '@tweet_deal_819',
 '@somedroidcom',
 '@weitingforyou',
 '@buy_now_811',
 '@shantelw6',
 '@hakanfagnell',
 '@new_offer_827',
 '@danowall',
 '@my3dprinting',
 '@reksaurian',
 '@lovepornsites',
 '@3dprintingjobs',
 '@trend_deal_420',
 '@hot_tips_123',
 '@triciaclyne',
 '@truthinautism',
 '@avs3dprint']

## 3. Normalization of The Tweets

In [22]:
df_selected = df[["Author ID", "Post Text", "Total Engagement", "Date", "tweet_id", "row_num"]]
df_selected["Normalized Text"] = df_selected["Post Text"].apply(normalize_post)
# Drop the intermediate column "Post Text" and keep its normalized version
df_selected = df_selected.drop(columns="Post Text")
df_selected

/tmp/ipykernel_938/4126986519.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected["Normalized Text"] = df_selected["Post Text"].apply(normalize_post)


,Author ID,Total Engagement,Date,tweet_id,row_num,Normalized Text
0,@Whimsu,0,2013-01-01 00:19:40+00:00,@Whimsu_2013-01-01 00:19:40+00:00,1,my sister got me this 3d printed gyroid for christmas! sooo cute!
1,@jonwitts,0,2012-12-30 20:18:03+00:00,@jonwitts_2012-12-30 20:18:03+00:00,2,"this is just so cool! "" @josiefraser : 3d printer converts digital audio files into 33rpm records http:// """
2,@MUAdvMfg,0,2012-12-30 15:13:18+00:00,@MUAdvMfg_2012-12-30 15:13:18+00:00,3,3d printing predictions for 2013 - http:// 229-3d-printing-predictions-for-2013.html …
3,@tttitoflip,0,2012-12-30 13:19:02+00:00,@tttitoflip_2012-12-30 13:19:02+00:00,4,features/ 3d printing http://
4,@3dprint_news,0,2012-12-30 22:37:16+00:00,@3dprint_news_2012-12-30 22:37:16+00:00,5,rt @ricardoblanco this is the best analysis of the pros and cons of commerical-scale 3d printing i've seen. http:// -achilles-heel-of-3d-printing-015281 …
...,...,...,...,...,...,...
5570575,@HelfrichBicycle,0,2022-12-17 04:56:50+00:00,@HelfrichBicycle_2022-12-17T04:56:50.000Z,6725981,"i custom built this wheel with a 3d printed logo for the @pmmarchingunit @pennmanorschools annual banquet basket raffle. #bicyclerepair #bicyclerestoration @ lancaster, pennsylvania"
5570576,@buyghostguns,0,2022-12-17 01:21:51+00:00,@buyghostguns_2022-12-17T01:21:51.000Z,6725982,sg22 .22lr 3d printed build kit w/cmmg parts https:// 2lr-3d-printed-build-kit-w-cmmg-parts/?utm_source=reviveoldpost&utm_medium=social&utm_campaign=reviveoldpost … #22lr #pistols
5570577,@DrMask12,13,2022-12-16 23:21:22+00:00,@DrMask12_2022-12-16T23:21:22.000Z,6725983,"finally decided to try and 3d print the b.p.s logo. need to tweak some settings and make it a little larger, but i'd say it's a great success. i made the 3d model of the logo, credit for the actual digital logo goes to @alexkisterr"
5570578,@ray5ar,0,2022-12-17 10:57:20+00:00,@ray5ar_2022-12-17T10:57:20.000Z,6725984,more 3d printed compliant mechanisms https:// eipa?feature=share … via @youtube


In [23]:
filtered = df_selected[df_selected['Normalized Text']
                       .str.contains(r'utm', case=False, na=False)]

# display it (in a notebook, Jupyter, etc.)
filtered

,Author ID,Total Engagement,Date,tweet_id,row_num,Normalized Text
77,@RicohRetail,2,2012-12-30 08:35:53+00:00,@RicohRetail_2012-12-30 08:35:53+00:00,90,doorbraak: iedereen kan eigen elpees maken #3d printing #rexp http:// lectronica/item/14652-doorbraak-iedereen-kan-eigen-elpees-maken?utm_source=facebook&utm_medium=social&utm_campaign=retail+detail+nederland#.un_8a9lcp_k.twitter …
88,@3DPrintingBuzz,0,2012-12-30 18:30:24+00:00,@3DPrintingBuzz_2012-12-30 18:30:24+00:00,102,a year in review: top ten stories in 3d printing http:// 0/a-year-in-review-top-ten-stories-in-3d-printing.html?utm_source=dlvr.it&utm_medium=twitter&utm_campaign=feed%3a+fabbaloo+%28fabbaloo%29#.uoch56wvhpu …
798,@bengoertzel,10,2012-12-29 16:59:13+00:00,@bengoertzel_2012-12-29 16:59:13+00:00,950,autodesk and organovo move toward easy 3d printing of organs -- http:// /autodesk-organovo-background/?utm_source=feedburner&utm_medium=feed&utm_campaign=feed%3a+wired%2findex+%28wired%3a+top+stories%29 …
842,@glundeen,0,2012-12-29 13:00:37+00:00,@glundeen_2012-12-29 13:00:37+00:00,1012,here's what a 3d-printed record sounds like [video] by @johnpaul http:// es-what-a-3d-printed-record-sounds-like-video?utm_source=feedburner&utm_medium=feed&utm_campaign=feed:+readwriteweb+(readwriteweb …
909,@MrDinoSossi,0,2012-12-29 17:14:11+00:00,@MrDinoSossi_2012-12-29 17:14:11+00:00,1096,"3d printing ""bigger than internet"" http:// 1/3d-printing-bigger-than-internet-?utm_source=taboola …"
...,...,...,...,...,...,...
5570333,@iNaor,0,2022-12-16 22:30:05+00:00,@iNaor_2022-12-16T22:30:05.000Z,6725707,printing atom by atom: lab explores nanoscale 3d printing https:// tom-lab-explores-nanoscale-3d.html?utm_source=dlvr.it&utm_medium=twitter … /via @physorg_com
5570356,@3DPrintingAgenc,0,2022-12-17 04:30:25+00:00,@3DPrintingAgenc_2022-12-17T04:30:25.000Z,6725734,"3d printed miniature: drennheim city - the cursed obelisk, tabletop rpg, highly detailed, gamer collectibles and gifts\n\ndirect link here: https:// eim-city-the-cursed-obelisk/?feed_id=10653&_unique_id=639d45e14d130&utm_source=twitter&utm_medium=ron%40webwaves.ca&utm_campaign=fs%20poster … https://"
5570370,@The_Derrick,0,2022-12-17 17:00:07+00:00,@The_Derrick_2022-12-17T17:00:07.000Z,6725749,"all sorts of homemade, crafty items, from ornament painting kits to ceramics to 3d-printed puzzles, will be decking the lobby of the franklin high school auditorium wednesday as the franklin school district holds its first art collection fundraiser. https:// e/franklin-students-getting-ready-for-first-art-show-fundraiser/article_6175b208-7d6b-11ed-8982-37a443431ff0.html?utm_campaign=blox&utm_source=twitter&utm_medium=social …"
5570372,@3DMetalPrintin1,0,2022-12-17 13:33:05+00:00,@3DMetalPrintin1_2022-12-17T13:33:05.000Z,6725751,"3d printing news briefs, december 17, 2022: paek patents, private flying vehicle, & more https:// ting-news-briefs-12-17-22/?utm_source=dlvr.it&utm_medium=twitter …"


### 3.1 Further Cleaning...Remove Remnant of URLs that are trickish to remove

In [24]:
df_selected['Normalized Text'] = (
    df_selected['Normalized Text']
        .str.replace(
            r'\S*(?:[?&]utm_(?:source|campaign|medium|content)=[^\s]*)',
            '',
            regex=True
        )
        .str.replace(r'\s+', ' ', regex=True)
        .str.strip()
)


In [25]:
df_selected['Normalized Text'] = (
    df_selected['Normalized Text']
        .str.replace(r'\?utm_id=[^\s]*', '', regex=True)
        #.str.replace(r'\s+', ' ', regex=True)
        .str.strip()
)


In [26]:
filtered = df_selected[df_selected['Normalized Text']
                       .str.contains(r'utm', case=False, na=False)]
filtered

,Author ID,Total Engagement,Date,tweet_id,row_num,Normalized Text
2521,@3DThinkTank,0,2012-12-26 20:25:28+00:00,@3DThinkTank_2012-12-26 20:25:28+00:00,3085,#bandsthatputmeinagoodmood have their own playlist and get blared on my 3d printed ipod amplifier. http://
2893,@OccupyTesla,0,2012-12-25 12:40:56+00:00,@OccupyTesla_2012-12-25 12:40:56+00:00,3550,<links updated> ultralight fractal structures may imitate and/or outmatch the strength of steel and be 3d printed ... http://
5547,@realbrendanb,0,2012-12-19 20:03:36+00:00,@realbrendanb_2012-12-19 20:03:36+00:00,6858,"3d printing + digital mapping is next level, props to cutmaps: http://"
17788,@RosmoRobot,0,2012-12-02 10:39:12+00:00,@RosmoRobot_2012-12-02 10:39:12+00:00,22076,rt @putmanbart working to market 3d printing gift to cosumers. #3dprinting : working to market 3d printi... http:// #3dprint
24953,@marekCzarzbon,0,2012-11-26 15:09:02+00:00,@marekCzarzbon_2012-11-26 15:09:02+00:00,30646,the future - 3d print - how it works - https:// player_embedded&v=qjutm0y7u1k# … ! http://
...,...,...,...,...,...,...
5560621,@hand71gljbn,0,2022-12-18 14:34:14+00:00,@hand71gljbn_2022-12-18T14:34:14.000Z,6714894,"flowers on black background blackout curtains,skull butterfly crown 3d printed drapes,2 panels window curtains,darkening curt tebc2vm https:// tag=alejandragutmann-20 …"
5560735,@Gambody_com,3,2022-01-16 18:03:06+00:00,@Gambody_com_2022-01-16T18:03:06.000Z,6715021,"you can 3d print the jurassic park diorama with stl files from gambody: https:// michael thomas purmenský put his utmost effort into rendering it in the most realistic way - the base has natural elements, and the cables on the posts are made out of real wire"
5561849,@esun3dfilament,9,2022-10-26 03:51:14+00:00,@esun3dfilament_2022-10-26T03:51:14.000Z,6716263,#3dprinted hygrometer stands by @strautmann_fotografie didn't they turn out pretty? printer: qidi tech i-mate s filament: esun abs+ . . #3dprinting #3dprinter #3dprint #esun #abs #hygrometer #hygrometerthermometer #thingiversemodel #3dmodels #3dmaker #3dprintinglife
5563364,@sq4dbuilds,11,2022-04-26 19:45:47+00:00,@sq4dbuilds_2022-04-26T19:45:47.000Z,6717978,3d printing footing walls saves time and money since no forms are needed as our concrete mix is stackable. #sq4d #makeitaboutme #tuesdayvibe #tuesdaymotivaton


## 4. Remove Empty Tweets/Posts
This could have happened if the author only posted a picture or video without text

In [27]:
df_selected = df_selected.sort_values(by="Normalized Text")
mask_empty = df_selected["Normalized Text"].str.strip().eq("")
df_empty = df_selected[mask_empty]
df_empty

,Author ID,Total Engagement,Date,tweet_id,row_num,Normalized Text
2368636,@Phrozen3D,0,2021-07-17 17:29:19+00:00,@Phrozen3D_2021-07-17T17:29:19.000Z,2765066,
2509416,@8BitoffunX,9,2021-12-03 19:45:00+00:00,@8BitoffunX_2021-12-03T19:45:00.000Z,2921320,
2296512,@JZKitty,9,2019-02-27 17:51:40+00:00,@JZKitty_2019-02-27T17:51:40.000Z,2685011,
5559814,@3dprintmars,0,2022-09-03 22:20:43+00:00,@3dprintmars_2022-09-03T22:20:43.000Z,6714001,
2457807,@Reksaurian,2,2021-11-29 16:20:52+00:00,@Reksaurian_2021-11-29T16:20:52.000Z,2864016,
...,...,...,...,...,...,...
5256904,NaN,0,2023-05-27 08:10:55+00:00,nan_2023-05-27T08:10:55.000Z,6380501,
5256903,NaN,0,2023-05-27 08:00:42+00:00,nan_2023-05-27T08:00:42.000Z,6380500,
5256901,NaN,0,2023-05-27 08:04:43+00:00,nan_2023-05-27T08:04:43.000Z,6380498,
2502595,@Puchiluh,199,2021-04-24 19:28:45+00:00,@Puchiluh_2021-04-24T19:28:45.000Z,2913846,


In [28]:
# Keep only rows where "Normalized Text" column is not empty string
df_selected = df_selected[~mask_empty]
df_selected

,Author ID,Total Engagement,Date,tweet_id,row_num,Normalized Text
2079733,@8BitoffunX,2,2019-08-08 02:08:04+00:00,@8BitoffunX_2019-08-08T02:08:04.000Z,2443875,!
4705938,@TmHessman,0,2013-12-27 13:56:41+00:00,@TmHessman_2013-12-27T13:56:41.000Z,5711826,! ! ! ! ! @build3dprinter: first working 3d printed liver expected by 2014 -- http:// --
4127193,@medovinabsinthe,0,2014-09-29 19:09:30+00:00,@medovinabsinthe_2014-09-29T19:09:30.000Z,4916844,"! "" @tuncakman : 3d printing with sand using the power of the sun http:// via @techcrunch """
5250396,@drmikemyers,0,2023-11-11 17:07:31+00:00,@drmikemyers_2023-11-11T17:07:31.000Z,6373375,! #tuckercarlson #nasa #trump #whitehouse canada's largest metal additive manufacturing facility opens in kitchener https://
5285682,@drmikemyers,0,2023-11-14 10:02:24+00:00,@drmikemyers_2023-11-14T10:02:24.000Z,6411350,! #tuckercarlson #nasa #trump #whitehouse expanding waterloo's additive manufacturing prowess https://
...,...,...,...,...,...,...
3709107,@Ticketmaster_GR,0,2015-11-04 14:36:59+00:00,@Ticketmaster_GR_2015-11-04T14:36:59.000Z,4362321,"󾮍 η έκθεση που θα σας πάρει το μυαλό! pac-man, super mario, tetris, ψηφιακή τεχνολογία και τέχνη, 3d printing ... http://"
1413647,@dEngzHue,0,2016-07-21 10:12:31+00:00,@dEngzHue_2016-07-21T10:12:31.000Z,1666199,"󾮜 @p280 only !! 󾬏new arrival u.s. style 3d print silk combined cotton blouse󾬏 󾔏cotton fabric, right thickness,... http://"
647011,@dEngzHue,0,2016-06-30 04:10:50+00:00,@dEngzHue_2016-06-30T04:10:50.000Z,758347,"󾮜@p380 only !! 󾬏new arrival u.s. style 3d print chiffon batwing sleeve polo shirt󾬏 󾔏chiffon fabric, right... http://"
1413423,@dEngzHue,0,2016-07-21 09:50:22+00:00,@dEngzHue_2016-07-21T09:50:22.000Z,1665946,󾮜@p380 only !! 󾬏new arrival u.s. style vivid 3d print spandex-chiffon like jumpsuit󾬏 󾔏spandex-chiffon like... http://


In [29]:
####--Helper function
def ascii_normalize_text(text):
    # Convert any non-ASCII fullwidth forms into ASCII equivalents
    # e.g., "ｔ" -> "t", "　" -> " ", etc.
    return unicodedata.normalize("NFKC", str(text))


## 5. Tweets with No Alphabetic Content
Rmeove tweets where there no letters, no digits, just things like !, ..., #@!, etc.

In [30]:
df_selected["Normalized Text"] = df_selected["Normalized Text"].apply(ascii_normalize_text)
mask_only_punct = df_selected["Normalized Text"].str.strip().apply(
    lambda x: bool(re.match(r'^[^\w]+$', x))  # "start ^, one or more non-alphanumeric chars, end $"
)

# This finds rows where 'Normalized Text' is purely punctuation/symbols 
df_only_punct = df_selected[mask_only_punct]

df_only_punct

,Author ID,Total Engagement,Date,tweet_id,row_num,Normalized Text
2079733,@8BitoffunX,2,2019-08-08 02:08:04+00:00,@8BitoffunX_2019-08-08T02:08:04.000Z,2443875,!
2476638,@Puchiluh,0,2021-02-17 22:29:43+00:00,@Puchiluh_2021-02-17T22:29:43.000Z,2884820,!!
1049523,@joeltelling,21,2017-04-17 22:46:11+00:00,@joeltelling_2017-04-17T22:46:11.000Z,1234724,!!!
822975,@joeltelling,10,2016-09-30 04:32:24+00:00,@joeltelling_2016-09-30T04:32:24.000Z,966210,!!!
2541531,@Reksaurian,2,2021-11-06 17:12:50+00:00,@Reksaurian_2021-11-06T17:12:50.000Z,2956760,!!!
5161208,@joeltelling,11,2023-03-03 16:45:28+00:00,@joeltelling_2023-03-03T16:45:28.000Z,6277958,!!!!
5447869,@3dprintmars,0,2022-05-14 19:19:39+00:00,@3dprintmars_2022-05-14T19:19:39.000Z,6590089,!!!!
5547646,@Shrimpy_shift11,0,2022-06-18 09:43:02+00:00,@Shrimpy_shift11_2022-06-18T09:43:02.000Z,6700591,!!!!!!!!!! !!!!!!!!!!!!!!!!!!!!! !!!!!!!!!!!!!!!!!!!!!!!!!!!!
2031124,@3DPrintRockets,1,2019-05-29 19:23:37+00:00,@3DPrintRockets_2019-05-29T19:23:37.000Z,2389751,#
1195913,@onlinewebshop0,0,2017-09-09 08:19:02+00:00,@onlinewebshop0_2017-09-09T08:19:02.000Z,1409222,#


In [31]:
# Remove those rows by inverting the mask with '~'
df_selected = df_selected[~mask_only_punct]
df_selected

,Author ID,Total Engagement,Date,tweet_id,row_num,Normalized Text
4705938,@TmHessman,0,2013-12-27 13:56:41+00:00,@TmHessman_2013-12-27T13:56:41.000Z,5711826,! ! ! ! ! @build3dprinter: first working 3d printed liver expected by 2014 -- http:// --
4127193,@medovinabsinthe,0,2014-09-29 19:09:30+00:00,@medovinabsinthe_2014-09-29T19:09:30.000Z,4916844,"! "" @tuncakman : 3d printing with sand using the power of the sun http:// via @techcrunch """
5250396,@drmikemyers,0,2023-11-11 17:07:31+00:00,@drmikemyers_2023-11-11T17:07:31.000Z,6373375,! #tuckercarlson #nasa #trump #whitehouse canada's largest metal additive manufacturing facility opens in kitchener https://
5285682,@drmikemyers,0,2023-11-14 10:02:24+00:00,@drmikemyers_2023-11-14T10:02:24.000Z,6411350,! #tuckercarlson #nasa #trump #whitehouse expanding waterloo's additive manufacturing prowess https://
5214050,@drmikemyers,0,2023-12-22 17:33:05+00:00,@drmikemyers_2023-12-22T17:33:05.000Z,6334461,! #tuckercarlson #nasa #trump #whitehouse the roboticworx universal controller is a 3d-printed acid-trip gadget for taking charge of the iot
...,...,...,...,...,...,...
3709107,@Ticketmaster_GR,0,2015-11-04 14:36:59+00:00,@Ticketmaster_GR_2015-11-04T14:36:59.000Z,4362321,"󾮍 η έκθεση που θα σας πάρει το μυαλό! pac-man, super mario, tetris, ψηφιακή τεχνολογία και τέχνη, 3d printing ... http://"
1413647,@dEngzHue,0,2016-07-21 10:12:31+00:00,@dEngzHue_2016-07-21T10:12:31.000Z,1666199,"󾮜 @p280 only !! 󾬏new arrival u.s. style 3d print silk combined cotton blouse󾬏 󾔏cotton fabric, right thickness,... http://"
647011,@dEngzHue,0,2016-06-30 04:10:50+00:00,@dEngzHue_2016-06-30T04:10:50.000Z,758347,"󾮜@p380 only !! 󾬏new arrival u.s. style 3d print chiffon batwing sleeve polo shirt󾬏 󾔏chiffon fabric, right... http://"
1413423,@dEngzHue,0,2016-07-21 09:50:22+00:00,@dEngzHue_2016-07-21T09:50:22.000Z,1665946,󾮜@p380 only !! 󾬏new arrival u.s. style vivid 3d print spandex-chiffon like jumpsuit󾬏 󾔏spandex-chiffon like... http://


## 6. Filtering Out Tweets That Contain No Alphabetic Characters (Unicode-Aware)
detecting rows with zero letters in any language, thanks to the \p{L} approach.

In [32]:
import regex

pattern = r'[\p{L}]'  # \p{L} matches any kind of letter from any language

mask_no_letters_v2 = df_selected["Normalized Text"].str.strip().apply(
    lambda x: not bool(regex.search(pattern, x))  # Use regex.search, not re.search
)

df_no_letters = df_selected[mask_no_letters_v2]
df_no_letters

,Author ID,Total Engagement,Date,tweet_id,row_num,Normalized Text
3034770,@Positiv3Prints,0,2018-12-14 18:09:10+00:00,@Positiv3Prints_2018-12-14T18:09:10.000Z,3520749,#2020
5449125,@3DPrintGeneral,46,2022-11-17 19:44:36+00:00,@3DPrintGeneral_2022-11-17T19:44:36.000Z,6591492,$699
5441363,@3dprintmars,0,2022-07-21 16:01:45+00:00,@3dprintmars_2022-07-21T16:01:45.000Z,6582920,$800!!!!!!!
1311917,@joeltelling,8,2017-09-13 20:15:51+00:00,@joeltelling_2017-09-13T20:15:51.000Z,1547105,+1
1260397,@Puchiluh,2,2017-08-20 07:02:18+00:00,@Puchiluh_2017-08-20T07:02:18.000Z,1486386,+10000000000000000000000
327143,@Puchiluh,0,2020-10-23 20:27:48+00:00,@Puchiluh_2020-10-23T20:27:48.000Z,398250,._.
4130339,@3DPrintRockets,0,2014-05-24 03:02:05+00:00,@3DPrintRockets_2014-05-24T03:02:05.000Z,4921338,00:00
563885,@3DPProfessor,8,2020-01-05 21:41:07+00:00,@3DPProfessor_2020-01-05T21:41:07.000Z,661232,01000011 01101100 01100101 01110110 01100101 01110010 00100000 01100001 01101110 01100100 00100000 01100001 01100100 01101111 01110010 01100001 01100010 01101100 01100101 00100001
2285283,@Puchiluh,17,2019-10-04 03:57:49+00:00,@Puchiluh_2019-10-04T03:57:49.000Z,2672446,05:55
2521463,@SMilloow,1,2021-05-29 01:26:40+00:00,@SMilloow_2021-05-29T01:26:40.000Z,2934531,100%


In [33]:
df_selected = df_selected[~mask_no_letters_v2]
df_selected

,Author ID,Total Engagement,Date,tweet_id,row_num,Normalized Text
4705938,@TmHessman,0,2013-12-27 13:56:41+00:00,@TmHessman_2013-12-27T13:56:41.000Z,5711826,! ! ! ! ! @build3dprinter: first working 3d printed liver expected by 2014 -- http:// --
4127193,@medovinabsinthe,0,2014-09-29 19:09:30+00:00,@medovinabsinthe_2014-09-29T19:09:30.000Z,4916844,"! "" @tuncakman : 3d printing with sand using the power of the sun http:// via @techcrunch """
5250396,@drmikemyers,0,2023-11-11 17:07:31+00:00,@drmikemyers_2023-11-11T17:07:31.000Z,6373375,! #tuckercarlson #nasa #trump #whitehouse canada's largest metal additive manufacturing facility opens in kitchener https://
5285682,@drmikemyers,0,2023-11-14 10:02:24+00:00,@drmikemyers_2023-11-14T10:02:24.000Z,6411350,! #tuckercarlson #nasa #trump #whitehouse expanding waterloo's additive manufacturing prowess https://
5214050,@drmikemyers,0,2023-12-22 17:33:05+00:00,@drmikemyers_2023-12-22T17:33:05.000Z,6334461,! #tuckercarlson #nasa #trump #whitehouse the roboticworx universal controller is a 3d-printed acid-trip gadget for taking charge of the iot
...,...,...,...,...,...,...
3709107,@Ticketmaster_GR,0,2015-11-04 14:36:59+00:00,@Ticketmaster_GR_2015-11-04T14:36:59.000Z,4362321,"󾮍 η έκθεση που θα σας πάρει το μυαλό! pac-man, super mario, tetris, ψηφιακή τεχνολογία και τέχνη, 3d printing ... http://"
1413647,@dEngzHue,0,2016-07-21 10:12:31+00:00,@dEngzHue_2016-07-21T10:12:31.000Z,1666199,"󾮜 @p280 only !! 󾬏new arrival u.s. style 3d print silk combined cotton blouse󾬏 󾔏cotton fabric, right thickness,... http://"
647011,@dEngzHue,0,2016-06-30 04:10:50+00:00,@dEngzHue_2016-06-30T04:10:50.000Z,758347,"󾮜@p380 only !! 󾬏new arrival u.s. style 3d print chiffon batwing sleeve polo shirt󾬏 󾔏chiffon fabric, right... http://"
1413423,@dEngzHue,0,2016-07-21 09:50:22+00:00,@dEngzHue_2016-07-21T09:50:22.000Z,1665946,󾮜@p380 only !! 󾬏new arrival u.s. style vivid 3d print spandex-chiffon like jumpsuit󾬏 󾔏spandex-chiffon like... http://


## 7. Drop Absolute Duplicates (!!!! This engagement based)

In [34]:
# Create a DataFrame of the duplicate rows that would be dropped
duplicates = df_selected[df_selected.duplicated(subset=["Normalized Text"], keep="first")]
print("Rows that would be dropped based on 'Normalized Text' duplicates:")
duplicates

Rows that would be dropped based on 'Normalized Text' duplicates:


,Author ID,Total Engagement,Date,tweet_id,row_num,Normalized Text
4541682,@RobertW79886085,0,2014-02-21 08:47:04+00:00,@RobertW79886085_2014-02-21T08:47:04.000Z,5493042,"! 3d printer creates creepy sculpture of your face in dark, milk or white chocolate http:// ?id_thread=64583 ..."
4541764,@Barbara07089370,0,2014-02-21 08:26:32+00:00,@Barbara07089370_2014-02-21T08:26:32.000Z,5493169,! 3d-printed eeg headset from openbci is customizable and open-source http:// ?id_thread=64401 ...
4455001,@Barbara23772984,0,2014-02-22 13:00:00+00:00,@Barbara23772984_2014-02-22T13:00:00.000Z,5374042,! 3d-printed hip implant lets teenager walk again http:// ?id_thread=67437 ...
3609006,@adaul989,0,2015-02-02 20:21:14+00:00,@adaul989_2015-02-02T20:21:14.000Z,4234760,! 3d-printed tumor replicas to better measure doses of cancer-fighting drugs on http://
3391938,@scoiel984,0,2015-03-28 22:11:51+00:00,@scoiel984_2015-03-28T22:11:51.000Z,3959028,! 3d-printed tumor replicas to better measure doses of cancer-fighting drugs on http://
...,...,...,...,...,...,...
1921757,@zivkelsey,0,2016-03-16 07:35:30+00:00,@zivkelsey_2016-03-16T07:35:30.000Z,2265666,"󾬏new arrival u.s. style rayon cotton 3d print vintage dress󾬏 󾔏rayon cotton fabric, with lining, 2-layer design,... http://"
798553,@phaulina,0,2016-06-14 14:28:48+00:00,@phaulina_2016-06-14T14:28:48.000Z,937087,"󾬏new arrival u.s. style rayon cotton 3d print vintage dress󾬏 󾔏rayon cotton fabric, with lining, 2-layer design,... http://"
1048315,@dEngzHue,0,2016-06-12 13:47:21+00:00,@dEngzHue_2016-06-12T13:47:21.000Z,1233318,"󾬏new arrival u.s. style rayon cotton 3d print vintage dress󾬏 󾔏rayon cotton fabric, with lining, 2-layer design,... http://"
736800,@dEngzHue,0,2016-03-10 17:21:56+00:00,@dEngzHue_2016-03-10T17:21:56.000Z,864757,"󾬏updated: 3d print cottony blouse these prints still avail, many stocks, feel free to update and resell :)󾬏 󾔏good... http://"


In [35]:
duplicates.shape

(1289477, 6)

In [36]:
# Sort by descending total engagement
df_selected = df_selected.sort_values("Total Engagement", ascending=False)

# Drop duplicates while keeping the rows with the highest engagement
df_selected = df_selected.drop_duplicates(subset=["Normalized Text"], keep="first")
df_selected

,Author ID,Total Engagement,Date,tweet_id,row_num,Normalized Text
1187670,@Bill_Gross,439944,2017-10-17 21:48:10+00:00,@Bill_Gross_2017-10-17T21:48:10.000Z,1399480,"in the ""i'm getting old"" department.., a kid saw this and said, ""oh, you 3d-printed the 'save' icon."""
2313605,@rustbeltlady,383384,2019-03-08 04:41:13+00:00,@rustbeltlady_2019-03-08T04:41:13.000Z,2703961,who gave my little brother a 3d printer
2479492,@McJesse,348608,2021-12-31 00:34:14+00:00,@McJesse_2021-12-31T00:34:14.000Z,2887953,"got a 3d printer for christmas, realized i can use it to print any new year’s glasses i want."
2547310,@olivelorraine_,283017,2021-08-15 20:47:32+00:00,@olivelorraine__2021-08-15T20:47:32.000Z,2963114,the vagina is the original 3d printer
2083066,@rveenewman,213595,2019-01-09 13:25:07+00:00,@rveenewman_2019-01-09T13:25:07.000Z,2447524,a 3d printed light projected animation. proof that there's always new ways to animate everything. #3dprint #animation
...,...,...,...,...,...,...
4815467,@BramKnaapen,0,2013-11-03 21:51:10+00:00,@BramKnaapen_2013-11-03T21:51:10.000Z,5849490,customizable 3d printed titanium glasses that look good; 3d printing is moving beyond gadgetry : http://
4315329,@SybilCollas,0,2014-01-17 15:31:11+00:00,@SybilCollas_2014-01-17T15:31:11.000Z,5178014,"customizable 3d printed tabletop miniatures. oh money, where are you when i need you so? http:// via @kickstarter"
4462367,@davebower,0,2014-01-16 20:40:39+00:00,@davebower_2014-01-16T20:40:39.000Z,5383868,customizable 3d printed tabletop miniatures. hmmmm. http:// orge/customizable-3d-printed-tabletop-miniatures ...
4582767,@SethRichard,0,2014-02-16 05:21:51+00:00,@SethRichard_2014-02-16T05:21:51.000Z,5550420,"customizable 3d printed tabletop miniatures, via @kickstarter only 46 hours left to back it https:// orge/customizable-3d-printed-tabletop-miniatures ..."


In [37]:
df_selected.shape

(4278932, 6)

## 8. Remove Tweets with Fewer Than N Alphabetic Characters and Zero Total Engagement
identifying and dropping tweets that are likely non-informative spam or noise (i.e. tweets with very few alphabetic characters and no engagement).

In [38]:
# Create a mask for tweets with fewer than 5 alphabetic characters
mask_few_letters = df_selected["Normalized Text"].apply(lambda text: sum(1 for c in text if c.isalpha()) < 10)

# Create a mask for tweets with Total Engagement equal to 0
mask_zero_engagement = df_selected["Total Engagement"] == 0

# Combine both conditions
combined_mask = mask_few_letters & mask_zero_engagement

# Show tweets that satisfy the condition
tweets_to_drop = df_selected[combined_mask]
print("Tweets that will be dropped:")
tweets_to_drop

# Now drop these tweets from df_selected
df_selected = df_selected[~combined_mask]


Tweets that will be dropped:


## 9. Pre-Identify English Lang Tweets
This is used to filter out tweets that are in English language but doesnt contain any keyword or term related to additive manufacturing

In [40]:
#from unidecode import unidecode
# === SETTINGS ===
CHUNK_SIZE = 100_000
MAX_WORKERS = 8

# === Detection logic ===
def detect_language_langdetect(text):
    try:
        #decoded = unidecode(text)
        return detect(text) == 'en'
    except:
        return False

# === Process a chunk ===
def process_chunk(chunk_df):
    with ProcessPoolExecutor(max_workers=MAX_WORKERS) as executor:
        results = list(tqdm(
            executor.map(detect_language_langdetect, chunk_df["Normalized Text"]),
            total=len(chunk_df),
            desc="LangDetect"
        ))
    chunk_df["langdetect_is_english"] = results
    return chunk_df

# === CHUNK IN MEMORY ===
def chunk_dataframe(df, chunk_size=100_000):
    for i in range(0, len(df), chunk_size):
        yield df.iloc[i:i + chunk_size]

# === MAIN ===
processed_chunks = []
for i, chunk in enumerate(chunk_dataframe(df_selected, chunk_size=CHUNK_SIZE)):
    print(f"Processing chunk {i + 1}")
    processed_chunk = process_chunk(chunk.copy())
    processed_chunks.append(processed_chunk)

# Combine all processed chunks into one final DataFrame
df_all = pd.concat(processed_chunks, ignore_index=True)


Processing chunk 1


LangDetect: 100%|█████████████████████████████████████████████████████████████████| 100000/100000 [00:46<00:00, 2159.17it/s]


Processing chunk 2


LangDetect: 100%|█████████████████████████████████████████████████████████████████| 100000/100000 [00:56<00:00, 1782.29it/s]


Processing chunk 3


...

LangDetect: 100%|█████████████████████████████████████████████████████████████████| 100000/100000 [01:23<00:00, 1199.20it/s]


Processing chunk 42


LangDetect: 100%|█████████████████████████████████████████████████████████████████| 100000/100000 [01:24<00:00, 1188.90it/s]


Processing chunk 43


LangDetect: 100%|███████████████████████████████████████████████████████████████████| 76611/76611 [01:07<00:00, 1137.96it/s]


In [41]:
df_all

,Author ID,Total Engagement,Date,tweet_id,row_num,Normalized Text,langdetect_is_english
0,@Bill_Gross,439944,2017-10-17 21:48:10+00:00,@Bill_Gross_2017-10-17T21:48:10.000Z,1399480,"in the ""i'm getting old"" department.., a kid saw this and said, ""oh, you 3d-printed the 'save' icon.""",True
1,@rustbeltlady,383384,2019-03-08 04:41:13+00:00,@rustbeltlady_2019-03-08T04:41:13.000Z,2703961,who gave my little brother a 3d printer,True
2,@McJesse,348608,2021-12-31 00:34:14+00:00,@McJesse_2021-12-31T00:34:14.000Z,2887953,"got a 3d printer for christmas, realized i can use it to print any new year’s glasses i want.",True
3,@olivelorraine_,283017,2021-08-15 20:47:32+00:00,@olivelorraine__2021-08-15T20:47:32.000Z,2963114,the vagina is the original 3d printer,True
4,@rveenewman,213595,2019-01-09 13:25:07+00:00,@rveenewman_2019-01-09T13:25:07.000Z,2447524,a 3d printed light projected animation. proof that there's always new ways to animate everything. #3dprint #animation,True
...,...,...,...,...,...,...,...
4276606,@BramKnaapen,0,2013-11-03 21:51:10+00:00,@BramKnaapen_2013-11-03T21:51:10.000Z,5849490,customizable 3d printed titanium glasses that look good; 3d printing is moving beyond gadgetry : http://,True
4276607,@SybilCollas,0,2014-01-17 15:31:11+00:00,@SybilCollas_2014-01-17T15:31:11.000Z,5178014,"customizable 3d printed tabletop miniatures. oh money, where are you when i need you so? http:// via @kickstarter",True
4276608,@davebower,0,2014-01-16 20:40:39+00:00,@davebower_2014-01-16T20:40:39.000Z,5383868,customizable 3d printed tabletop miniatures. hmmmm. http:// orge/customizable-3d-printed-tabletop-miniatures ...,True
4276609,@SethRichard,0,2014-02-16 05:21:51+00:00,@SethRichard_2014-02-16T05:21:51.000Z,5550420,"customizable 3d printed tabletop miniatures, via @kickstarter only 46 hours left to back it https:// orge/customizable-3d-printed-tabletop-miniatures ...",True


In [42]:
df_filtered = df_all[df_all['langdetect_is_english'] == False]
df_filtered 

,Author ID,Total Engagement,Date,tweet_id,row_num,Normalized Text,langdetect_is_english
35,@oemonur,37315,2022-09-19 18:30:35+00:00,@oemonur_2022-09-19T18:30:35.000Z,6574681,işten kovulmadan önce son maaşımla 3d printer almıştım. yaptığım en mantıklı yatırım olabilir. son bi haftadır deliler gibi dantel basıyorum.,False
48,@dr_chaku,26629,2020-03-25 15:06:14+00:00,@dr_chaku_2020-03-25T15:06:14.000Z,452635,terima kasih geng 3d printing malaysia sebab beri face shield 3d print secara percuma di hospital dan klinik seluruh malaysia. kami hargai!,False
61,@canitti,21371,2020-03-16 19:48:52+00:00,@canitti_2020-03-16T19:48:52.000Z,322795,"yardımlaşmayı öğreneceğiz... i̇talya'nın brescia bölgesinde bir hastane, yoğun bakımdaki 250 korona virüs hastası için solunum cihazına bağlayıcı vana tükenince 3d printer üreticisi bir firma, 2 gün uyumayıp birim maliyeti 1 euro'dan az şekilde 100 tane üretip hastaneye vermiş.",False
68,@printingguns,19855,2022-07-08 20:34:45+00:00,@printingguns_2022-07-08T20:34:45.000Z,6558904,i 3d printing,False
109,@Puchiluh,12782,2020-08-21 10:23:03+00:00,@Puchiluh_2020-08-21T10:23:03.000Z,604431,"en la plaza de la luna de madrid hay un establecimiento donde el dueño enseña una palabra en chino cada día en una pizarrita os parecerá una chorrada pero que todos los días el dueño se ponga a pensar con ilusión qué va a poner, es muy dulce",False
...,...,...,...,...,...,...,...
4276544,@signorina37H,0,2015-12-17 18:00:58+00:00,@signorina37H_2015-12-17T18:00:58.000Z,3927293,customizable 3d printed laser cut lamp http:// #rounders,False
4276545,@randomshandom,0,2015-12-17 17:24:16+00:00,@randomshandom_2015-12-17T17:24:16.000Z,3927268,customizable 3d printed laser cut lamp http:// #instructable,False
4276546,@universaltvstan,0,2015-12-17 17:29:33+00:00,@universaltvstan_2015-12-17T17:29:33.000Z,3927041,customizable 3d printed laser cut lamp http://,False
4276568,@Just_Print_It,0,2015-12-12 22:20:25+00:00,@Just_Print_It_2015-12-12T22:20:25.000Z,3887880,customizable alien egg http://,False


In [43]:
df_filtered_for_processing = df_filtered[["Normalized Text", "row_num"]]
df_filtered_for_processing.to_csv(f"non_english_tweets_{CURRENT_DATE_TIME}.csv", index=False)

In [44]:
df_selected = df_all #NOTE: anti-best practice

## 10. Tweets That Doesnt Contain Keywords and Has 0 Engagement

In [45]:
# ##----

# pattern = r"(?i)(#?additive(?:\s*manufacturing|maufacturing)?|#?3[-\s]*d|#?3[-\s]*d[-\s]*print(?:ed|ing|er)?|#?printing|#?metal)"
# # Create a mask for tweets that do NOT contain any of the keywords
# mask_no_keywords = ~df_selected["Normalized Text"].str.contains(pattern, regex=True)

# # Create a mask for tweets with Total Engagement equal to 0
# mask_zero_engagement = df_selected["Total Engagement"] == 0

# # Ensure langdetect flag is False (i.e., not English)
# mask_english = df_selected["langdetect_is_english"] == True

# # Combine all three conditions:
# # - No keywords
# # - Zero engagement
# # - Not detected as English
# mask_combined = mask_no_keywords & mask_zero_engagement & mask_english

# # Filter the DataFrame accordingly
# tweets_without_keywords_and_zero_engagement = df_selected[mask_combined]

# tweets_without_keywords_and_zero_engagement

In [46]:
# 1. Keep only tweets detected as English
df_selected = df_selected[df_selected['langdetect_is_english'] == True]

# Remove all (?i) from your patterns!
base_pattern = r"(#?additive(?:\s*manufacturing|maufacturing)?|#?3[-\s]*d|#?3[-\s]*d[-\s]*print(?:ed|ing|er)?|#?printing|#?metal)"

additional_patterns = [
    # photopolymer & resin processes
    r"(#?SLA|#?stereolithography|#?DLP|#?digital[-\s]*light[-\s]*processing)",
    # powder-based processes
    r"(#?SLS|#?selective[-\s]*laser[-\s]*sintering|#?SLM|#?selective[-\s]*laser[-\s]*melting|#?DMLS|#?direct[-\s]*metal[-\s]*laser[-\s]*sintering|#?EBM|#?electron[-\s]*beam[-\s]*melting|#?LPBF|#?powder[-\s]*bed[-\s]*fusion)",
    # jetting & droplet processes
    r"(#?MJF|#?multijet[-\s]*fusion|#?PolyJet|#?binder[-\s]*jetting|#?drop[-\s]*on[-\s]*demand)",
    # extrusion & filament processes
    r"(#?FDM|#?FFF|#?fused[-\s]*deposition|#?fused[-\s]*filament[-\s]*fabrication|#?FGF|#?fused[-\s]*granulate[-\s]*fabrication)",
    # materials & feedstocks
    r"(#?filament|#?resin|#?photopolymer|#?powder|#?nylon|#?PLA|#?ABS|#?PETG|#?polycarbonate)",

    # Synonyms & General AM Phrases
    r"(#?rapid[-\s]*prototyp(?:ing|e)|#?direct[-\s]*digital[-\s]*manufacturing|#?DDM|#?layer[-\s]*by[-\s]*layer|#?tool[-\s]*free[-\s]*manufacturing|#?generative[-\s]*manufacturing|#?digital[-\s]*fabrication)",

    # Sheet Lamination & Other Less-Common Processes
    r"(#?sheet[-\s]*lamination|#?LOM|#?laminated[-\s]*object[-\s]*manufacturing|#?ultrasonic[-\s]*additive|#?UAM|#?ultrasonic[-\s]*consolidation)",

    # Directed Energy Deposition (DED) Family
    r"(#?DED|#?directed[-\s]*energy[-\s]*deposition|#?laser[-\s]*metal[-\s]*deposition|#?LMD|#?laser[-\s]*cladding|#?wire[-\s]*arc[-\s]*additive|#?WAAM|#?electron[-\s]*beam[-\s]*additive|#?wire[-\s]*feed)",

    # Binder Jetting Variants & Brands
    r"(#?metal[-\s]*binder[-\s]*jetting|#?ceramic[-\s]*binder[-\s]*jetting|#?voxeljet|#?exone|#?sand[-\s]*printing|#?sand[-\s]*binder[-\s]*jet)",

    # Notable Brands & Commercial Terms
    r"(#?stratasys|#?ultimaker|#?formlabs|#?markforged|#?prusa|#?creality|#?makerbot|#?renishaw|#?EOS|#?desktopmetal|#?3dsystems|#?exone|#?voxeljet|#?SLM[-\s]*Solutions|#?arcam|#?GE[-\s]*Additive|#?HP[-\s]*Jet[-\s]*Fusion|#?3DSystems|#?HP3D|#?DesktopMetal)",

    # Polymers, Metals, and Materials
    r"(#?inconel|#?titanium|#?stainless[-\s]*steel|#?aluminum|#?cobalt[-\s]*chrome|#?carbon[-\s]*fiber|#?composite|#?ceramic|#?elastomer|#?TPU|#?peek|#?PEEK|#?PMMA|#?brass|#?bronze|#?ULTEM|#?alumina|#?thermoplastics|#?thermosets|#?composites|#?ceramics)",

    # Hybrid/Novel & Adjacent Processes
    r"(#?hybrid[-\s]*manufacturing|#?robotic[-\s]*additive|#?multi[-\s]*material|#?4D[-\s]*printing|#?bioprint(?:ing|er)?|#?concrete[-\s]*printing|#?food[-\s]*printing)",

    # Specific applications
    r"(#?bioprinting|#?medical[-\s]*3D[-\s]*printing|#?dental[-\s]*3D[-\s]*printing|#?aerospace[-\s]*3D[-\s]*printing|#?automotive[-\s]*3D[-\s]*printing|#?customization|#?on[-\s]*demand[-\s]*manufacturing)",

    # Software and design aspects
    r"(#?CAD|#?CAM|#?DFAM|#?topology[-\s]*optimization|#?slicing|#?gcode)",

    # Post-processing and finishing
    r"(#?postprocessing|#?surface[-\s]*finishing|#?powder[-\s]*removal)",

    # General industry hashtags
    r"(#?AMindustry|#?3Dprintingindustry|#?manufacturinginnovation|#?industry40)",


    ###--------NEEEWWWW
    # plain “prototype”, “prototypes”, or “prototyping”
    r"\bprototyp(?:e|es|ing)\b",

    # 3D scanning & reverse‐engineering
    r"\b3[-\s]*d[-\s]*scan(?:ning)?\b",
    r"\bscan(?:ner|ning)?\b",
    r"\breverse[-\s]*engineer(?:ing)?\b",

    # mesh & lattice structures
    r"\bmesh(?:es)?\b",
    r"\blattice\b",
    r"\bvoxel(?:s)?\b",

    # slicer & g-code
    r"\bslicer\b",
    r"\bgcode\b",

    # infill & supports
    r"\binfill\b",
    r"\bsupports?\b",
    r"\braft\b",
    r"\bbrim\b",
    r"\bskirt\b",

    # printer hardware
    r"\bnozzle\b",
    r"\bextruder\b",
    r"\blayer[-\s]*height\b",
    r"\boverhang\b",

    # photopolymer & resin processes
    r"(#?SLA|#?stereolithography|#?DLP|#?digital[-\s]*light[-\s]*processing)",
    # powder-based processes
    r"(#?SLS|#?selective[-\s]*laser[-\s]*sintering|#?SLM|#?selective[-\s]*laser[-\s]*melting|#?DMLS|#?direct[-\s]*metal[-\s]*laser[-\s]*sintering|#?EBM|#?electron[-\s]*beam[-\s]*melting|#?LPBF|#?powder[-\s]*bed[-\s]*fusion)",
    # jetting & droplet processes
    r"(#?MJF|#?multijet[-\s]*fusion|#?PolyJet|#?binder[-\s]*jetting|#?drop[-\s]*on[-\s]*demand)",
    # extrusion & filament processes
    r"(#?FDM|#?FFF|#?fused[-\s]*deposition|#?fused[-\s]*filament[-\s]*fabrication|#?FGF|#?fused[-\s]*granulate[-\s]*fabrication)",
    # materials & feedstocks
    r"(#?filament|#?resin|#?photopolymer|#?powder|#?nylon|#?PLA|#?ABS|#?PETG|#?polycarbonate)",
    # … (the rest of your existing patterns) …

    
]

# Combine all patterns into one big OR'd regex pattern
combined_pattern = base_pattern + "|" + "|".join(additional_patterns)

# Now pass the IGNORECASE flag to .str.contains()
mask_no_keywords = ~df_selected["Normalized Text"].str.contains(combined_pattern, regex=True, flags=re.IGNORECASE)
mask_zero_engagement = df_selected["Total Engagement"] == 0

mask_no_keywords_and_zero_engagement = mask_no_keywords & mask_zero_engagement

tweets_without_keywords_and_zero_engagement = df_selected[mask_no_keywords_and_zero_engagement]

df_selected = df_selected[~mask_no_keywords_and_zero_engagement]


/tmp/ipykernel_938/367972388.py:103: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_no_keywords = ~df_selected["Normalized Text"].str.contains(combined_pattern, regex=True, flags=re.IGNORECASE)


## Writeout tweets with no keywords to csv for review...

In [47]:
##----
# Write it out to csv for review
filename = f"no_keywords_tweets_{CURRENT_DATE_TIME}.csv"
tweets_without_keywords_and_zero_engagement.to_csv(filename, index=False)
# spammer_tweets.to_csv(filename, index=False)
# print(f"CSV file '{filename}' has been written.")
print(df_selected.shape)

(3922038, 7)


## Write the final cleaned dataframe to csv

In [48]:
# Create filename
filename = f"CLEANED_data_pre_near_duplicates_handled_{CURRENT_DATE_TIME}.csv"

# # Filter duplicates
# df_selected = df_selected[ddf_selected.duplicated(subset="Normalized Texts", keep=False)]
# print("Duplicates found:")
# print(duplicates)

# Save to CSV
df_selected.to_csv(filename, index=False)


In [ ]:
def count_column_tokens_two(df, column_name, model_name="gpt-4o") -> int:
    """
    Count the total number of tokens in df[column_name] using the tokenizer for model_name.
    Suppresses errors for special tokens like <|endoftext|>.
    """
    import tiktoken

    try:
        enc = tiktoken.encoding_for_model(model_name)
    except KeyError:
        enc = tiktoken.get_encoding("cl100k_base")

    texts = df[column_name].fillna("").astype(str)

    # Suppress special token errors by allowing all special tokens
    token_counts = texts.apply(lambda txt: len(enc.encode(txt, disallowed_special=())))

    return int(token_counts.sum())

In [ ]:
num_tokens = count_column_tokens_two(df_selected, "Normalized Text", model_name="gpt-4o")

print(f"Total tokens in 'Normalized Text' column: {num_tokens}")

In [ ]:
print("==============COMPLETED===============")